# GTP로 이미지 설명 만들기

In [21]:
import base64
from openai import OpenAI
from src.utils import get_openai_api_key

In [22]:
# OpenAI 객체 생성
client = OpenAI(api_key=get_openai_api_key())

**Lorem Picsum** 이미지 설명 요청

In [23]:
# 이미지 설명을 요청하는 메시지 프롬프트 작성
messages = [
    {
        'role': 'user',
        'content': [
            {'type': 'text', 'text': '이 이미지를 설명해줘.'},
            {'type': 'image_url',
             'image_url': {'url': 'https://fastly.picsum.photos/id/22/4434/3729.jpg?hmac=fjZdkSMZJNFgsoDh8Qo5zdA_nSGUAWvKLyyqmEt2xs0'}},
        ],
    },
]

In [24]:
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=messages
)

In [25]:
response

ChatCompletion(id='chatcmpl-CDldE46SYPEekK461k9egwZ1xFJ8k', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='이미지는 한 남성이 도로 위를 걷고 있는 장면을 보여줍니다. 그는 정장을 입고 있으며, 하늘색 넥타이를 매고 있습니다. 도로는 비어 있으며, 흰색 선들이 그어진 바닥이 보입니다. 전체적인 분위기는 차분하고 약간 고요한 느낌을 줍니다. 남성은 걷고 있는 방향을 바라보며, 주변은 거의 인적이 없습니다.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1757397396, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_e665f7564b', usage=CompletionUsage(completion_tokens=98, prompt_tokens=25515, total_tokens=25613, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [26]:
response.choices[0].message.content

'이미지는 한 남성이 도로 위를 걷고 있는 장면을 보여줍니다. 그는 정장을 입고 있으며, 하늘색 넥타이를 매고 있습니다. 도로는 비어 있으며, 흰색 선들이 그어진 바닥이 보입니다. 전체적인 분위기는 차분하고 약간 고요한 느낌을 줍니다. 남성은 걷고 있는 방향을 바라보며, 주변은 거의 인적이 없습니다.'

# 로컬 이미지 파일의 설명 요청하기

* 이미지 바이너리 데이터를 텍스트 형식으로 변환해서 GPT chat.completions 요청을 보내야 함.
* base64 라이브러리 : 이진(binary) 데이터를 아스키(ASCII) 문자로 인코딩(변환).
    * 전송/저장 호환성
    * 데이터 손상 방지
    * 프로토콜 호환성

In [27]:
def base64encode_image(image_file):
    with open(file=image_file, mode='rb') as f:  # 이미지 파일을 오픈.
        data = f.read()  # 이미지 파일의 이진 데이터를 읽음.

        # 이진 데이터를 base64 인코딩을 해서 ASCII 문자열로 변환, UTF-8로 변환해서 리턴.
        return base64.b64encode(data).decode(encoding='utf-8')

In [28]:
image_path_1 = './images/cafe1.jpg'
image_path_2 = './images/cafe2.jpg'

In [29]:
base64_image_1 = base64encode_image(image_path_1)

In [30]:
base64_image_1[:100]    # base64로 인코딩 된 문자열들 중에서 앞에서 100개만 출력

'/9j/4AAQSkZJRgABAgEASABIAAD/4gxYSUNDX1BST0ZJTEUAAQEAAAxITGlubwIQAABtbnRyUkdCIFhZWiAHzgACAAkABgAxAABh'

In [31]:
# GPT에게 보낼 메시지 프롬프트 작성
messages = [
    {
        'role': 'user',
        'content': [
            {'type': 'text', 'text': '이미지에 대해서 설명해줘.'},
            {
                'type': 'image_url',
                'image_url': {'url': f'data:image/jpeg;base64,{base64_image_1}'}
            },
        ],
    },
]

In [32]:
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=messages
)

In [34]:
response.choices[0].message.content

'이미지는 현대적인 카페 또는 레스토랑 내부를 보여줍니다. 공간은 큰 창과 유리 천장으로 자연광이 가득 차 있으며, 다양한 식물들이 천장과 벽에 걸려 있습니다. 중앙에는 긴 나무 테이블과 여러 개의 둥근 테이블이 배치되어 있고, 사람들이 함께 앉아 대화하는 모습이 보입니다. 바닥은 콘크리트로 되어 있으며, 바 뒤에는 바와 주방이 보이는 구조입니다. 전체적으로 아늑하고 세련된 분위기를 자아내고 있습니다.'

In [35]:
base64_image_2 = base64encode_image(image_path_2)  # cafe2.jpg를 base64 인코딩

In [36]:
base64_image_2[:100]

'/9j/4AAQSkZJRgABAgEASABIAAD/4gxYSUNDX1BST0ZJTEUAAQEAAAxITGlubwIQAABtbnRyUkdCIFhZWiAHzgACAAkABgAxAABh'

In [37]:
# 이미지 비교를 요청하는 메시지 프롬프트 작성
messages = [
    {
        'role': 'user',
        'content': [
            {'type': 'text', 'text': '두 카페의 차이점을 설명해줘.'},
            {
                'type': 'image_url',
                'image_url': {'url': f'data:image/jpeg;base64,{base64_image_1}'},
            },
            {
                'type': 'image_url',
                'image_url': {'url': f'data:image/jpeg;base64,{base64_image_2}'},
            },
        ],
    },
]

In [38]:
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=messages
)

In [39]:
print(response.choices[0].message.content)

두 카페의 차이점을 몇 가지로 설명해 드릴게요.

1. **장식 및 분위기**:
   - 첫 번째 카페는 식물 장식이 많이 되어 있고, 자연광이 많이 들어오는 공간으로, 생기있고 쾌적한 느낌을 줍니다.
   - 두 번째 카페는 더 모던하고 산업적인 느낌을 주며, 나무와 금속 소재의 조화로 따뜻하면서도 세련된 분위기를 자아냅니다.

2. **배치 및 좌석**:
   - 첫 번째 카페는 긴 테이블과 다양한 크기의 좌석이 있어 대화와 모임에 적합해 보입니다.
   - 두 번째 카페는 개인적인 공간을 고려하여 배치가 이루어져 있어, 혼자서 작업하거나 조용히 시간을 보내기 좋은 환경입니다.

3. **조명**:
   - 첫 번째 카페는 자연광을 강조한 반면, 두 번째 카페는 조명 기구가 눈에 띄어 인공 조명에 의존하는 느낌이 강합니다.

4. **전체적인 느낌**:
   - 첫 번째 카페는 친환경적이고 여유로운 느낌을 주는 반면, 두 번째 카페는 트렌디하고 도시적인 느낌이 강조됩니다. 

이런 요소들이 두 카페의 매력을 다르게 만들어 줍니다.
